# Clustering con K-Means. Sistema recomendador de películas.
Eres un analista de datos en Netflix y quieres analizar los gustos de las personas basándote en la votación que le han dado a las diferentes películas que han visto.

¿Es posible construir un sistema recomendador de películas para nuestros usuarios?
- Vamos a utilizar el algoritmo de Clustering K-Means para generar distintos grupos/clústers de usuarios con gustos parecidos, basándonos en las votaciones que han dado a las películas que han visto.
- Despues utilizaremos los resultados obtenidos para recomendar películas que no haya visto un determinado usuario, pero que sí hayan visto otros usuarios dentro de su mismo clúster de gustos.


Usaremos el dataset de [MovieLens](https://movielens.org/) [user rating dataset](http://files.grouplens.org/datasets/movielens/ml-latest-small.zip). 

## Importar y analizar los datos
Obtendremos los datos de dos ficheros:
- movies.csv: Contiene un listado de películas y los géneros de cada película.
- ratings.csv: Contiene un listado de usuarios con la votación que han hecho a distintas películas.
    
Vamos a unificar estos ficheros para tener toda la información en un solo dataset. Primero importaremos los dos ficheros en dos dataframes, despues uniremos los dos dataframes.

Nota 1: Para leer ficheros con pandas usaremos [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

Nota 2: Con la función [merge()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) podemos unir dos dataframes (funciona igual que el join de SQL)

In [ ]:
# Importamos las librerías necesarias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
import helper
import pickle

In [ ]:
# Importamos el dataset de películas.
movies = pd.read_csv('data/movies.csv')
movies.head()

In [ ]:
# Importamos el dataset de las votaciones
ratings = pd.read_csv('data/ratings.csv')
ratings.head()

In [ ]:
print('Nuestro dataset contiene: ', len(ratings), ' votaciones de ', len(movies), ' películas.')

In [ ]:
# Unimos los dos datasets usando movieId, para tener en cada fila 
#la votación de cada usuario junto a los datos de la película votada.
user_movies_ratings = ratings.merge(right=movies, how='left', on='movieId')
#user_movies_ratings[merged_ratings['userId']==64]
user_movies_ratings.head()

¿Cuántas filas tendrá nuestro dataset unificado?

In [ ]:
# TODO

# ----

## Funcionamiento K-Means: Romance vs. Scifi
Para entender como funciona [K-Means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html), vamos a centrarnos en un subconjunto de usuarios. Analizaremos sus preferencias de géneros. 

Para ahorrar tiempo, usaremos algunas funciones que nos ayudarán a preprocesar los datos. Las podrás encontrar en el fichero `helpers.py`.

Nota: Usaremos la función `get_genre_ratings(ratings, genres, column_names)` para obtener una media de las votaciones de cada usuario para los géneros que indiquemos.

In [ ]:
# Calculamos la media de votaciones para los géneros Romance y Ciencia Ficción
genre_ratings = helper.get_genre_ratings(user_movies_ratings, ['Romance', 'Sci-Fi'], ['avg_romance_rating', 'avg_scifi_rating'])
genre_ratings.head()

La función `get_genre_ratings` ha calculado, para cada usuario, la media de todas las votaciones que ha hecho de las películas de cada uno de los géneros.

Vamos a user la función `bias_genre_rating_dataset()` para descartar los usuarios a los que les gustan los dos géneros, para ver más claramente la separación por gustos.

In [ ]:
dataset = genre_ratings
biased_dataset = helper.bias_genre_rating_dataset(genre_ratings, 3.2, 2.5)
print( "Número de registros: ", len(biased_dataset))

### Visualización de datos

Vamos a imprimir los datasets:

In [ ]:
%matplotlib inline

helper.draw_scatterplot(dataset['avg_scifi_rating'],'Avg scifi rating', dataset['avg_romance_rating'], 'Avg romance rating')
helper.draw_scatterplot(biased_dataset['avg_scifi_rating'],'Avg scifi rating', biased_dataset['avg_romance_rating'], 'Avg romance rating')


A simple vista podemos observar la separación de los datos. ¿Cómo ve los datos el algoritmo K-Means?
Vamos a ejecutarlo para que intente separar los datos en dos grupos (K=2)

### Ejecutar K-Means

In [ ]:
# Importamos KMeans
from sklearn.cluster import KMeans

# Creamos una instancia de KMeans para encontrar 2 clusters y ejecutamos la funcion fit para generar el modelo.
kmeans_1 = KMeans(n_clusters=2, random_state=0).fit(biased_dataset[['avg_scifi_rating','avg_romance_rating']])
# Ejecutamos predict para clusterizar el dataset
predictions = kmeans_1.predict(biased_dataset[['avg_scifi_rating','avg_romance_rating']])

# Usamos la función draw_clusters para imprimir graficamente el resultado
helper.draw_clusters(biased_dataset, predictions, ['romance','scifi'])

Podemos ver que el algoritmo ha tenido en cuenta principalmente la valoración de las películas de romance. Si la puntuación media es mayor a 3 estrellas, entonces el usuario pertenece al primer grupo, si es menor pertenece al otro grupo.

¿Y si probamos a dividirlo en tres grupos?

In [ ]:
# Creamos una instancia de KMeans para encontrar 3 clusters
kmeans_2 = KMeans(n_clusters=3, random_state=1)

# Ejecutamos fit_predict para clusterizar el dataset
predictions_2 = kmeans_2.fit_predict(biased_dataset[['avg_scifi_rating','avg_romance_rating']])

# Usamos la función draw_clusters para imprimir graficamente el resultado
helper.draw_clusters(biased_dataset, predictions_2, ['romance','scifi'])

Ahora vemos como las puntuaciones de ciencia ficción están teniendo más relevancia para el algoritmo.
* Usuarios a los que les gusta el romance pero no la ciencia ficción (grupo morado).
* Usuarios a los que les gusta la ciencia ficción pero no el romance (grupo amarillo) 
* Usuarios a los que les gusta la ciencia ficción y el romance
 

### Ejercicio
* Intenta clusterizar el dataset original, en lugar del que hemos filtrado. Puedes probar con K=2 y K=3
* Hemos comparado los géneros de ciencia ficción y romance. Prueba a comparar otros dos géneros. Para obtener la lista de géneros, puedes usar la función `helper.get_genres()`

In [ ]:
# TODO

# ----

## Sistema recomendador de películas
Hemos visto como funciona K-Means con las votaciones para dos géneros. Vamos a volver a nuestro Sistema Recomendador de Películas. Para ello tendremos en cuenta todas las votaciones de todas las películas para crear nuestros clusters.

### Tratando con valores nulos.
Vamos a crear una pivot table con la siguiente forma:
- filas -> usuarios
- columnas -> peliculas
- celda (fila, columna) -> valoración de un determinado usuario (fila) para una determinada película (columna).

In [ ]:
user_movie_ratings = pd.pivot_table(user_movies_ratings, index='userId', columns= 'title', values='rating')

print('Tenemos %s filas (usuarios) y %s columnas (películas) ' % (user_movie_ratings.shape[0], user_movie_ratings.shape[1]))
user_movie_ratings.head()

Nos encontramos con un problema muy frecuente: La existencia de valores nulos. Esto quiere decir que muchos usuarios no han votado muchas películas. Esto es normal porque hay muchas películas que no han visto.

Nos interesa tener todos los datos concentrados en una zona de la tabla, así que vamos a ordenar la tabla para tener, al principio de la tabla, los datos de las películas con más votaciones y los usuarios que han votado mayor número de películas.

Para ello vamos a usar la función `sort_by_rating_density` que nos ordenará la tabla tal y como hemos descrito arriba.

In [ ]:
n_movies = 30
n_users = 20
most_rated_movies_users_selection = helper.sort_by_rating_density(user_movie_ratings, n_movies, n_users)
most_rated_movies_users_selection.head()

Esto tiene mejor pinta. Hemos condensado toda la información útil en la zona superior de nuestro dataset. Ahora podemos visualizarlo.

Para visualizar grandes cantidades de datos podemos utilizar mapas de calor. Nuestra función `draw_movies_heatmap` nos ayudará con esto. Veamos como se ven los datos una vez ordenados.

In [ ]:
helper.draw_movies_heatmap(most_rated_movies_users_selection)

Cada columna es una película. Cada fila es un usuario. El color de cada celda indica la valoración del usuario para ésa película.

Las celdas blancas indican que el usuario no ha votado esa película todavía.
A KMeans no le gustan los valores vacíos, así que los reemplazaremos antes de empezar el entrenamiento.

## Clusterizando películas
Con K-Means, tenemos que especificar el número de clústers K. Vamos a probar con 20.

Recordemos que a K-Means no le gustan los valores nulos, así que usaremos la función `fillna` de pandas para reemplazar los valores nulos por 0.

In [ ]:
#user_movie_ratings =  pd.pivot_table(ratings_title, index='userId', columns= 'title', values='rating')
# Por motivos de recursos, trabajaremos con 1000 películas en vez de las +9000
most_rated_movies_1k = helper.get_most_rated_movies(user_movie_ratings, 1000)
most_rated_movies_1k

In [ ]:
# K-Means con 20 clusters
kmeans_20 = KMeans(n_clusters=20, algorithm='full', random_state=2).fit(most_rated_movies_1k.fillna(0))
# Calculamos los clusters para todos los registros de nuestro dataset)
predictions = kmeans_20.predict(most_rated_movies_1k.fillna(0))
# También podemos calcular el clúster de un solo registro.
prediction = kmeans_20.predict(most_rated_movies_1k.fillna(0).iloc[0:1])


In [ ]:
max_users = 70 # max_users para imprimir mapas de calor
max_movies = 50 # max_movies para imprimir mapas de calor
# Concatenamos las predicciones a nuestro data set, en la columna grupo.
most_rated_movies_1k_clustered = pd.concat([most_rated_movies_1k.reset_index(), pd.DataFrame({'group':predictions})], axis=1)
most_rated_movies_1k_clustered
# Podemos imprimir los mapas de calor de cada uno de los clústers.
#helper.draw_movie_clusters(most_rated_movies_1k_clustered, max_users, max_movies)

## Predicción

Ya tenemos nuestro modelo capaz de clusterizar y decirnos a que grupo pertenece un usuario a partir de sus votaciones.
Veamos a ver qué podemos hacer con nuestro modelo. 
* Vamos a seleccionar un clúster y algún usuario dentro de ese cluster. 
* Después identificaremos las películas que no ha visto dentro de ese clúster.
* Sabemos que, dentro de ese clúster, los usuarios tienen gustos comunes.
* Intentaremos predecir la puntuación que ese usuario le dará a la película cuando la vea.

In [ ]:
# Selecciona un ID de los clústers de arriba (columna grupo)
cluster_number = 5

# Filtramos para ver los datos de ese cluster
n_users = 70
n_movies = 50
cluster = most_rated_movies_1k_clustered[most_rated_movies_1k_clustered.group == cluster_number]
# ordenados con los valores más altos en la parte alta de la tabla
cluster = helper.sort_by_rating_density(cluster.drop(['index', 'group'], axis=1), n_movies, n_users)
helper.draw_movies_heatmap(cluster, axis_labels=False)
cluster.fillna('')[:10]

Selecciona una celda en blanco de la tabla. Está en blanco porque el usuario no ha votado la película todavía.

¿Podemos predecir si le gustará a nuestro usuario la película?

Como el usuario esta en un grupo con gustos similares, podemos calcular la media de las puntuaciones de los demás usuarios para esa película en ese cluster. Obtendremos la valoración aproximada que le daría este usuario tras ver la película.



In [ ]:
movie_name = 'Truman Show, The (1998)'
print('Nuestro algoritmo predice que el usuario le dará a %s una recomendación de: %s ' % (movie_name, str(cluster[movie_name].mean())) )

## Recomendaciones
¡Ya estamos cerca!
Vamos a repasar lo que hemos hecho:
- Hemos usado K-Means para clusterizar usuarios según las puntuaciones que le han dado a distintas películas.
- Esto nos ha generado clusters con puntuaciones similares y, como consecuencia, con gustos cinéfilos similares.

Teniendo en cuesta lo anterior, nuestro sistema de recomendación funcionará de la siguiente manera:

- Cuando encontremos un usuario que no ha puntuado una película, podemos calcular la media de las puntuaciones de los demás usuarios en el cluster.
- También podemos calcular la media de puntuaciones de cada película dentro de un clúster, para saber cuanto gusta esa película dentro de ese clúster.



Vamos a calcular la puntuación media de las 20 películas más votadas de nuestro cluster

In [ ]:
# putuación media de las 20 primeras películas
cluster.mean().head(20).sort_values(ascending=False)

Esto es muy útil, porque podemos usarlo en nuestro sistema de recomendación, lo cual permitirá a nuestros usuarios descubrir películas que probablemente les gustarán.


In [ ]:
cluster2 = most_rated_movies_1k_clustered[most_rated_movies_1k_clustered.group == cluster_number]
cluster2.fillna('')[:20]

In [ ]:
#Seleccionamos un usuario de la tabla de arriba, la que obtuvimos al ejecutar el comando 'cluster.fillna('').head()'
user_id = 21
# Obtenemos todas las peliculas que ha votado el usuario
user_2_ratings =cluster2.iloc[user_id,:]
user_2_ratings[user_2_ratings.notna()]

# Películas que no ha votado el usuario
user_2_unrated_movies =  user_2_ratings[user_2_ratings.isnull()]
# Concatenamos la lista de películas que no ha votado el usuario y la lista de puntuaciones medias del cluster
# para obtener las votaciones de las películas que no ha visto el usuario
avg_ratings = pd.concat([user_2_unrated_movies, cluster.mean()], axis=1, join='inner').loc[:,0]

# Ordenamos ascendentemente según puntuaciones
avg_ratings.sort_values(ascending=False)[:20]

Estas son las recomendaciones para nuestro usuario!

## Exportar modelo

In [ ]:
filename = 'kmeans_20.sav'
pickle.dump(kmeans_20, open(filename, 'wb'))

## Importar modelo

In [ ]:
model = pickle.load(open(filename, 'rb'))
model.predict(most_rated_movies_1k.fillna(0))